# Grafik

In [44]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

kolam = pd.read_excel('../data/fishbook_dataset.xlsx', 'Dataset_Kolam_1')
kolam

,Hari Kalendar,Hari Kalender,Populasi,Kematian,ABW (g),ADG (g),Feeding Rate,Pakan,pH Pagi,pH Sore,...,Nitrit,D.O. pagi,D.O. sore,Salinitas,Suhu pagi,Suhu sore,Amonia,Alkalinitas,Kadar Mg,Kadar Ca
0,2022-07-31,0,1000,NaN,94.9,NaN,0.1,9490.00,7.7,7.8,...,0.1,5.5,7.5,34,28.0,25.0,0.0,97,1050,315
1,2022-08-01,1,995,5.0,95.4,0.5,0.1,9492.30,7.6,7.8,...,0.1,5.3,7.5,34,28.1,27.0,0.0,125,950,285
2,2022-08-02,2,994,1.0,96.2,0.8,0.1,9562.28,7.6,7.8,...,NaN,5.3,7.5,34,28.7,27.0,0.0,135,1094,500
3,2022-08-03,3,993,1.0,96.6,0.4,0.1,9592.38,7.6,7.8,...,NaN,5.3,7.4,34,27.3,28.5,0.0,120,1050,315
4,2022-08-04,4,988,5.0,97.5,0.9,0.1,9633.00,7.6,7.8,...,0.2,5.3,7.4,33,27.2,28.5,0.0,125,1203,400
5,2022-08-05,5,981,7.0,98.9,1.4,0.1,9702.09,7.6,7.8,...,0.2,5.3,7.4,33,25.7,26.0,0.0,120,900,270
6,2022-08-06,6,978,3.0,99.0,0.1,0.1,9682.20,7.6,7.8,...,NaN,5.4,7.4,32,28.0,27.0,0.1,115,1162,420
7,2022-08-07,7,973,5.0,99.5,0.5,0.1,9681.35,7.6,7.8,...,0.7,5.4,7.4,32,28.0,29.0,0.0,115,1093,470
8,2022-08-08,8,972,1.0,100.6,1.1,0.1,9778.32,7.6,7.8,...,0.6,5.2,7.2,32,28.0,29.0,0.1,120,1059,350
9,2022-08-09,9,969,3.0,103.1,2.5,0.1,9990.39,7.5,7.8,...,NaN,5.2,7.2,31,26.0,28.0,0.0,135,1075,295


In [45]:
# Metrics FCR

fig = go.Figure(go.Indicator(
    mode = "number+delta", # valuenya SUM dari pakan dibagi 10, terus berat akhir dikurangi berat awal dikali populasi awal
    value = round((list(kolam['Pakan'].cumsum())[-1] / 10) / ((kolam.iloc[-1,4] - kolam.iloc[0,4]) * 1000),2),
    title = {"text": "FCR<br><span style='font-size:25em;color:gray'>"},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig.update_layout(paper_bgcolor = "lightgray")

fig.show()

In [46]:
# Pie Chart Survival Rate
svr = {'ket': ['mati', 'hidup'], 'pop': [kolam.iloc[0,2] - kolam.iloc[-1, 2], kolam.iloc[-1,2]]}
svr = pd.DataFrame(svr)

fig = go.Figure(data=[go.Pie(labels=svr['ket'], values=svr['pop'], hole=.5)])
fig.update_layout(
    #title_text="Survival Rate",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text=str(round((kolam.iloc[-1,2]/kolam.iloc[0,2])*100, 2))+'%', x=0.5, y=0.5, font_size=40, showarrow=False)])
fig.show()

In [47]:
# Metrics Total Pakan
a = (list(kolam['Pakan'].cumsum())[-1] - kolam.iloc[-1,7]) / 1000

fig = go.Figure(go.Indicator(
    mode = "number+delta", # valuenya SUM dari pakan dibagi 10, terus berat akhir dikurangi berat awal dikali populasi awal
    value = round((list(kolam['Pakan'].cumsum())[-1] / 1000),2),
    title = {"text": "Total Pakan (kg)<br><span style='font-size:25em;color:gray'>"},
    domain = {'x': [0, 1], 'y': [0, 1]},
    delta = {'reference': a, 'relative': True}))

fig.update_layout(paper_bgcolor = "lightgray")

fig.show()

In [48]:
# Grafik Pertumbuhan ABW

def pred_abw(n):
  y = 0.0016 * (n ** 2) + 0.0261 * (n) - 0.1265
  return y

pred_x = [kolam.iloc[-1,1]+1, kolam.iloc[-1,1]+2, kolam.iloc[-1,1]+2]
pred_y = []

for i in pred_x:
  y = pred_abw(i)
  pred_y.append(y)

fig = go.Figure()
fig.add_trace(go.Scatter(x=kolam['Hari Kalender'], y=kolam['ABW (g)'],
                    mode='lines+markers',
                    name='Rataan Berat (gram)'))
fig.add_trace(go.Scatter(x=pred_x, y=pred_y,
                    mode='markers', name='Prediksi'))
fig.update_layout(title='Rataan Berat Harian dan Prediksinya (gram)',
                   xaxis_title='Hari Kultur',
                   yaxis_title='Berat (gr)')
fig.show()

In [49]:
# Grafik ADG
fig = px.line(kolam, x="Hari Kalender", y="ADG (g)", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [50]:
# Grafik Pakan Kumulatif
fig = px.line(kolam, x="Hari Kalender", y=kolam["Pakan"].cumsum(), width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [51]:
# Grafik Pakan Harian

fig = px.line(kolam, x="Hari Kalender", y="Pakan", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [52]:
# Grafik pH pagi

fig = px.line(kolam, x="Hari Kalender", y="pH Pagi", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [53]:
# Grafik pH sore

fig = px.line(kolam, x="Hari Kalender", y="pH Sore", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [54]:
# Grafik Nitrit

fig = px.line(kolam, x="Hari Kalender", y="Nitrit", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [55]:
# Grafik D.O. pagi

fig = px.line(kolam, x="Hari Kalender", y="D.O. pagi", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [56]:
# Grafik D.O. sore

fig = px.line(kolam, x="Hari Kalender", y="D.O. sore", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [57]:
# Grafik Salinitas

fig = px.line(kolam, x="Hari Kalender", y="Salinitas", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [58]:
# Grafik Suhu pagi

fig = px.line(kolam, x="Hari Kalender", y="Suhu pagi", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [59]:
# Grafik Suhu sore

fig = px.line(kolam, x="Hari Kalender", y="Suhu sore", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [60]:
# Grafik Amonia

fig = px.line(kolam, x="Hari Kalender", y="Amonia", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [61]:
# Grafik Alkalinitas

fig = px.line(kolam, x="Hari Kalender", y="Alkalinitas", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [62]:
# Grafik Kadar Mg

fig = px.line(kolam, x="Hari Kalender", y="Kadar Mg", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

In [63]:
# Grafik Kadar Ca

fig = px.line(kolam, x="Hari Kalender", y="Kadar Ca", width=500, height=400)
fig.update_traces(textposition="bottom right")
fig.show()

# Kalkulator

In [65]:
# Kalkulator
# Input
wt = 1 # kg
ct = 7 # berapa ind ikan yg disampel
mrt = 2 # ikan yg mati
fd_rt = 0.1 #10% # feeding rate dari tabel

ABW = wt / ct # rata-rata berat ikan
Size = 1/ABW # berapa ikan dalam 1 kg
ADG = (wt - kolam['ABW (g)'][-1]) # berapa pertumbuhan BB per hari
surv_rt = (kolam['Populasi'][-1] - mrt)/kolam['Populasi'][0] # (pop_akhir - yg mati hari ini) / pop_awal
fd_pd = ABW * fd_rt * kolam['Populasi'][-1] # berapa pakan per hari, rata BB * feeding rate * pop_akhir
bmss = (1/fd_rt) * fd_pd
pop  = bmss / ABW

KeyError: -1

In [66]:
from datetime import datetime as dt

inp_Date = str(input()) # input tanggal, format YYYY-mm-dd
inp_Datetm = dt.strptime(inp_Date, '%Y-%m-%d')
inp_Date_ = dt.date(inp_Datetm)
date_bfr = kolam.iloc[-1,0]

delta = inp_Date_ - date_bfr
delta.days # Ini untuk Hari Kultur

wt = float(input())
ct = int(input())
ABW = wt / ct # input tabel
ADG = (ABW - kolam.iloc[-1,4]) # input tabel
mortal = int(input())  # input tabel
survived = kolam.iloc[-1,2] - mortal # input tabel
fd_rate = float(input()) # input tabel
pakan = survived * fd_rate * ABW # input tabel
pH_pagi = float(input())
pH_sore = float(input())
range_pH = pH_sore - pH_pagi
Nitrit = float(input())
DO_pagi = float(input())
DO_sore = float(input())
Salin = float(input())
Suhu_pagi = float(input())
Suhu_sore = float(input())
Amon = float(input())
Alkali = float(input())
Kadar_Mg = float(input())
Kadar_Ca = float(input())


ValueError: time data '1' does not match format '%Y-%m-%d'

In [ ]:
update =

In [ ]:
d = {'a':[1,3], 'b':[2,5], 'c':[3,4]}
d = pd.DataFrame(d)
a = {'a':3, 'c':2, 'b':3}
d = d.append(a, ignore_index=True)
d

# Buat masukin data ke tabel

In [ ]:
# Panen
parsial = int(input()) # input ke tabel
Panen_Date = str(input())
tgl = dt.strptime(Panen_Date, '%Y-%m-%d')
Pan_Date = dt.date(tgl) # input ke tabel
ABW = int(input())
Populasi = int(input())
Perc_pop = Populasi / kolam.iloc[0,2]